# Tools and Routing

In [ ]:
#!pip install wikipedia
#!pip install openapi-pydantic
#!pip install langchain-community==0.3.1
import json
import boto3
import panel as pn  # GUI

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [ ]:
from langchain_core.tools import tool

In [ ]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [ ]:
search.name

In [ ]:
search.description

In [ ]:
search.args

In [ ]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [ ]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [ ]:
search.args

In [ ]:
search.run("sf")

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.name

In [ ]:
get_current_temperature.description

In [ ]:
get_current_temperature.args

In [ ]:
get_current_temperature.run({"latitude": 37.7749, "longitude": -122.4194})

In [ ]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia.name

In [ ]:
search_wikipedia.description

In [ ]:
search_wikipedia.run({"query": "langchain"})

## Using langgraph syntax since APIChain is deprecated.

See https://api.python.langchain.com/en/latest/_modules/langchain/chains/api/base.html#APIChain.from_llm_and_api_docs

In [ ]:
from langchain_aws import ChatBedrock
from langchain.chains.api.prompt import API_URL_PROMPT
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper

In [ ]:
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

url3 = "https://petstore3.swagger.io/api/v3/openapi.json"
text = requests.get(url3).text

In [ ]:
toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),  # no auth required
    allow_dangerous_requests=True,
)
tools = toolkit.get_tools()

In [ ]:
api_request_chain = (
    API_URL_PROMPT.partial(api_docs=text)
    | model.bind_tools(tools, tool_choice="any")
)

In [ ]:
api_request_chain.invoke({"question":"What are three pets names?"})

In [ ]:
api_request_chain.invoke({"question":"Tell me about pet with id 42"})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [ ]:
model_with_tools = model.bind_tools(tools=[search_wikipedia, get_current_temperature])

In [ ]:
model_with_tools.invoke("what is the weather in sf right now")

In [ ]:
model_with_tools.invoke("what is langchain")

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. Use tools when possible. Make best guesses about city locations."),
    ("user", "{input}"),
])
chain = prompt | model_with_tools

In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
from langchain.agents.output_parsers import ToolsAgentOutputParser

In [ ]:
chain = prompt | model_with_tools | ToolsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "What is the weather in san francisco right now"})

In [ ]:
result

In [ ]:
resultTool = result[0]
resultTool.tool

In [ ]:
resultTool.tool_input

In [ ]:
get_current_temperature(resultTool.tool_input)

In [ ]:
result = chain.invoke({"input": "hi!"})

In [ ]:
type(result)

In [ ]:
result

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result[0].tool].run(result[0].tool_input)

In [ ]:
chain = prompt | model_with_tools | ToolsAgentOutputParser() | route

In [ ]:
result = chain.invoke({"input": "What is the weather in san francisco right now"})

In [ ]:
result

In [ ]:
result = chain.invoke({"input": "What is langchain?"})

In [ ]:
result

In [ ]:
chain.invoke({"input": "hi!"})